In [27]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Importo librería de Carla

from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None) # Muestra todas las columnas de los DataFrame
pd.set_option('display.width', 1000) # Ajusta el ancho de la consola para evitar truncamientos


In [48]:
# Creo una función que recibe la URL de una película en IMDB y devuelve un diccionario con los datos de sus 10 intérpretes principales.

def conseguir_datos_actores (url_pelicula):
    # Activo Selenium y abro un navegador para explorar la web de la película.
    driver = webdriver.Chrome()
    driver.get(url_pelicula)
    print("Entrando en la url de la película.")
    sleep(2)
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    print("Aceptando cookies.")
    sleep(2)
    print("Maximizo ventana.")
    sleep(2)

    # Inicializo el diccionario donde guardaré las URL y el nombre de cada intérprete.

    actores_url = {} 
    lista_actores = []

    # Busco el elemento con la información del casting al completo y lo itero en busca del nombre y la url de cada intérprete.

    elemento_casting = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid")
    print("Explorando el casting")
    sleep(2)

    for i in range (1, 11):
        try:
            actor = elemento_casting.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.kSFMrr > a")
            nombre_actor = actor.text
            # nombre.append(nombre_actor)
            actores_url[nombre_actor] = actor.get_attribute("href")
        except:
            print("No se ha encontrado o no se ha podido extraer la url de algún actor. Seguimos ejecutando con normalidad.")
    print("Extraigo el nombre y url de los actores")
    sleep(2)

    # Entramos en las webs de los actores para extraer sus datos

    for actor, url in actores_url.items():
        print(f"Entrando en la url de {actor}")
        driver.get(url)
        sleep(2)

        print("\tExtraemos año de nacimiento")
        fecha_completa = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-11.hFWIYv > section > aside > div > span:nth-child(2)").text
        lista_year = fecha_completa.split()
        year = lista_year[-1]

        print(f"\tExtraemos otros roles en la industria")
    
        sleep(2)
        lista_roles = []
        roles= driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div > ul").find_elements("css selector", "li")
        for rol in roles:
            lista_roles.append(rol.text)
            
        sleep(2)

        print("\tRecopilando otras películas por las que se le conoce")

        conocido_por = [] 

        selector_conocido_por = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-1c5b3162-0.dsjiSY > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid")
        otras_peliculas = selector_conocido_por.find_elements("css selector", "a") # Busca todos los enlaces que haya en la caja de conocido por.
        for pelicula in otras_peliculas:
            titulo = pelicula.text
            if len(titulo)>1:                   # Esta condición es para evitar que capture títulos vacíos.
                conocido_por.append(titulo)     
        

        print("\tRecopilando premios")
        # Al estar en la versión MPV, nos vamos a quedar solo con el resumen de los premios.

        premios = []

        try: 
            resumen_premios = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__label.ipc-metadata-list-item__label--link").text
            premios.append(resumen_premios)
            sleep(2)
        except:
            print(f"{actor} no tiene resumen de premios")
            premios.append("Sin premios relevantes")
        
        tupla_actor = (actor, year, lista_roles, conocido_por, resumen_premios)
        lista_actores.append(tupla_actor)       
    
    print("Finalizamos exploración de los actores principales")
    driver.quit()
    print("Cerrando navegador")
    print(lista_actores)
    return lista_actores

mi_lista = conseguir_datos_actores("https://www.imdb.com/title/tt0107290/?ref_=nv_sr_srsg_0_tt_6_nm_2_in_0_q_jur")



Entrando en la url de la película.
Aceptando cookies.
Maximizo ventana.
Explorando el casting
Extraigo el nombre y url de los actores
Entrando en la url de Sam Neill
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Laura Dern
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Jeff Goldblum
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Richard Attenborough
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le conoce
	Recopilando premios
Entrando en la url de Bob Peck
	Extraemos año de nacimiento
	Extraemos otros roles en la industria
	Recopilando otras películas por las que se le cono

In [49]:
df = pd.DataFrame(mi_lista)
print(df)

                      0     1                                           2                                                  3                                       4
0             Sam Neill  1947            [Reparto, Producción, Dirección]  [Jurassic Park (Parque Jurásico), A la caza de...  Nominado para 2 premios Primetime Emmy
1            Laura Dern  1967                [Reparto, Producción, Guion]  [Jurassic Park (Parque Jurásico), Historia de ...                     Ganó 1 premio Óscar
2         Jeff Goldblum  1952                [Reparto, Producción, Guion]  [Jurassic Park (Parque Jurásico), La mosca, El...            Nominado para 1 premio Óscar
3  Richard Attenborough  1923            [Reparto, Producción, Dirección]  [Jurassic Park (Parque Jurásico), La gran evas...                    Ganó 2 premios Óscar
4              Bob Peck  1945                                   [Reparto]  [Jurassic Park (Parque Jurásico), Edge of Dark...                     Ganó 1 premio BAFTA
5        M